# 3-2 Grammars & Entities

## Imports, Custom Function, Data

In [ ]:
# IMPORTS
import re
import nltk


# Improved normalization function
# Removes all punctuation except periods (for sentences)
# Lowercases all words and tokenizes
def tknize (a_string):
    # Handle all the string operations at one time
    clean = re.sub('[^a-zA-Z \.]', ' ', a_string).lower()
    # Create the list of sub-strings (tokens) and return it
    return nltk.tokenize.word_tokenize(clean)

# DATA
with open("../data/mdg.txt", mode="r", encoding="utf-8") as f:
            mdg = f.read()

print(mdg[0:50])

"Off there to the right -- somewhere -- is a large


In [24]:
# Break them into a list of tokens
mdg_ = tknize(mdg)

print(mdg_[0:10])

['off', 'there', 'to', 'the', 'right', 'somewhere', 'is', 'a', 'large', 'island']


## Parsing

See the [list of PoS tags](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) on the UPenn website for details on tags. For a complete list, run the following: `nltk.help.upenn_tagset()` -- please make sure you download the tag set first, `nltk.download("tagsets")`. 

In [26]:
# mdg_ is our cleaned list of tokens which keeps only periods.
mdg_tagged = nltk.pos_tag(mdg_)

mdg_tagged[0:10]

[('off', 'RB'),
 ('there', 'EX'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('right', 'NN'),
 ('somewhere', 'RB'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('large', 'JJ'),
 ('island', 'NN')]

QUIZ: Describe the data structure above.

In [23]:
# You can re-assemble this to see what the parser is going to see
reassembled = [tagged[0] for tagged in mdg_tagged[0:10]]
print(" ".join(reassembled))

off there to the right somewhere is a large island


The purpose of the code block above is to demonstrate that the tagged document is simply a list of tuples, which you can manipulate a number of ways, here I am simply grabbing the first value in the tuple, `tuple[0]`. 

We could also do something like this:

In [8]:
for t in mdg_tagged[0:100]:
    if t[1] == 'JJ':
        print(t[0])

large
rainsford
old
ship
trap
suggestive
curious
i
dank
tropical
palpable
thick
warm
good


***Uh oh!*** NLTK isn't infallible! The NLTK has tagged "rainsford" and "i" as adjectives. (Tagging the "I" as an adjective is something I have encountered before.) First, you should know there are alternatives to NLTK -- I recommend spaCy over TextBlob -- but you also have to determine if these are one-off errors that you can ignore or if you can ignore these errors because of the scale of your work. This is an analytical judgment call: you must make it for yourself, but you must **document** it for others. *Documentation makes **you** look smart!*

### Textblob

In [11]:
# Get the functionality we need
from textblob import TextBlob

# Create the blob
blob = TextBlob(re.sub('[^a-zA-Z \.]', ' ', mdg).lower())

# Get the noun phrases (so easy!)
mdg_np = blob.noun_phrases

# See some results
for i in mdg_np[0:10]:
    print(i)

large island
old charts
ship trap island whitney
suggestive name isn t
curious dread
i don t
dank tropical night
warm blackness
ve good eyes
i ve


***Lions, tigers, and bear! Oh my!*** TextBlob produces weird results as well. This is not easy! (Think about the implications for large language models and how many inaccurate results are being "averaged" out.)

### spaCy

In [14]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_)


Apple Apple PROPN NNP
is be AUX VBZ
looking look VERB VBG
at at ADP IN
buying buy VERB VBG
U.K. U.K. PROPN NNP
startup startup VERB VBD
for for ADP IN
$ $ SYM $
1 1 NUM CD
billion billion NUM CD


In [18]:
mdg_doc = nlp(mdg)

for token in mdg_doc[0:10]:
    print(token.text, token.pos_)

" PUNCT
Off ADV
there ADV
to ADP
the DET
right NOUN
-- PUNCT
somewhere ADV
-- PUNCT
is AUX


In [21]:
mdg_PPNs = []
for token in mdg_doc:
    if token.pos_ == "PROPN":
        mdg_PPNs.append(token.text)

print(set(mdg_PPNs))

{'London', 'Puritan', 'liqueur', 'Ugh', 'Swamp', 'Death', 'East', 'weary', 'borsch', 'Aurelius', 'astrakhan', 'Pol', 'Doubtless', 'Purdey', 'City', 'Rains', 'revoir', 'Watch', 'Great', 'Mr.', 'Malacca', 'Sanger', 'jut', 'Laughter', 'Uganda', 'gray', 'Americans', 'Cliquot', 'Tibet', 'Splendid', 'Marcus', 'God', 'Lucar', 'Folies', 'Pistol', 'bush', 'Gasping', 'Guns', 'Rainsford', 'Boredom', 'San', 'Carlo', 'General', 'Chambertin', 'Amazon', 'Sleep', 'Sea', 'Veuve', 'Bah', 'Cossack', 'French', 'Trap', 'Ivan', 'America', 'Russian', 'Ganges', 'I.', 'Zaroff', 'Caucasus', '.the', 'Russians', 'ford', 'Providence', 'Monte', "anything?'--", 'York', 'huskily', 'Africa', 'Butterfly', 'cr^epes', 'Bergere', 'Nielsen', 'bay', 'china', 'Island', 'general', 'Whitney', 'Captain', 'Roger', 'Swam', 'Dusk', 'English', 'Ship', 'Chablis', 'Lazarus', 'Moscow', 'Swede', 'au', 'American', 'New', 'Czar', 'Madame', 'Russia', 'khaki', 'White', 'staccato', 'Cape', 'Chinese', 'Mirage', 'Automatically', 'Suzette', 'n

In [27]:
# Compare NLTK and Spacy on proper nouns
mdg_PPNs_nltk = []
for token in mdg_tagged:
    if token[1] == "NNP":
        mdg_PPNs_nltk.append(token[0])

print(mdg_PPNs_nltk)

['.the', 'rainsford', 'rainsford', 'rainsford', 'general', 'rainsford']


## Chunking

In [10]:
# How to Read the Grammar below:
# Start with an optional (?) determiner ('DT')
# Can have any number (*) of adjectives (JJ)
# End with a noun (<NN>)
grammar = "NP: {<DT>?<JJ>*<NN>}"

# Instantiate the chunk parser
parser = nltk.RegexpParser(grammar)

# Run it on our tagged text
tree = parser.parse(mdg_tagged)

# See some results
for i in tree[0:20]:
    print(i)

('off', 'RB')
('there', 'EX')
('to', 'TO')
(NP the/DT right/NN)
('somewhere', 'RB')
('is', 'VBZ')
(NP a/DT large/JJ island/NN)
('said', 'VBD')
(NP whitney/NN)
('.', '.')
('it', 'PRP')
('s', 'VBZ')
('rather', 'RB')
(NP a/DT mystery/NN)
('what', 'WP')
(NP island/NN)
('is', 'VBZ')
('it', 'PRP')
('rainsford', 'JJ')
('asked', 'VBD')


In [11]:
print(tree[6])

(NP a/DT large/JJ island/NN)


In [12]:
print(len(tree), tree.height())

7722 3


In [13]:
print(tree.treepositions()[0:20])

[(), (0,), (1,), (2,), (3,), (3, 0), (3, 1), (4,), (5,), (6,), (6, 0), (6, 1), (6, 2), (7,), (8,), (8, 0), (9,), (10,), (11,), (12,)]


In [14]:
NPtrees = [subtree for subtree in tree if type(subtree) == nltk.Tree and subtree.label() == "NP"]

print(len(NPtrees))
print(NPtrees[0:20])

1593
[Tree('NP', [('the', 'DT'), ('right', 'NN')]), Tree('NP', [('a', 'DT'), ('large', 'JJ'), ('island', 'NN')]), Tree('NP', [('whitney', 'NN')]), Tree('NP', [('a', 'DT'), ('mystery', 'NN')]), Tree('NP', [('island', 'NN')]), Tree('NP', [('ship', 'JJ'), ('trap', 'JJ'), ('island', 'NN')]), Tree('NP', [('whitney', 'NN')]), Tree('NP', [('a', 'DT'), ('suggestive', 'JJ'), ('name', 'NN')]), Tree('NP', [('isn', 'NN')]), Tree('NP', [('a', 'DT'), ('curious', 'JJ'), ('dread', 'NN')]), Tree('NP', [('the', 'DT'), ('place', 'NN')]), Tree('NP', [('some', 'DT'), ('superstition', 'NN')]), Tree('NP', [('rainsford', 'NN')]), Tree('NP', [('the', 'DT'), ('dank', 'JJ'), ('tropical', 'JJ'), ('night', 'NN')]), Tree('NP', [('thick', 'JJ'), ('warm', 'JJ'), ('blackness', 'NN')]), Tree('NP', [('the', 'DT'), ('yacht', 'NN')]), Tree('NP', [('whitney', 'NN')]), Tree('NP', [('a', 'DT'), ('laugh', 'NN')]), Tree('NP', [('i', 'NN')]), Tree('NP', [('a', 'DT'), ('moose', 'NN')])]


In [15]:
NPleaves = [subtree.leaves() for subtree in tree if type(subtree) == nltk.Tree and subtree.label() == "NP"]

print(len(NPleaves))
print(NPleaves[0:20])

1593
[[('the', 'DT'), ('right', 'NN')], [('a', 'DT'), ('large', 'JJ'), ('island', 'NN')], [('whitney', 'NN')], [('a', 'DT'), ('mystery', 'NN')], [('island', 'NN')], [('ship', 'JJ'), ('trap', 'JJ'), ('island', 'NN')], [('whitney', 'NN')], [('a', 'DT'), ('suggestive', 'JJ'), ('name', 'NN')], [('isn', 'NN')], [('a', 'DT'), ('curious', 'JJ'), ('dread', 'NN')], [('the', 'DT'), ('place', 'NN')], [('some', 'DT'), ('superstition', 'NN')], [('rainsford', 'NN')], [('the', 'DT'), ('dank', 'JJ'), ('tropical', 'JJ'), ('night', 'NN')], [('thick', 'JJ'), ('warm', 'JJ'), ('blackness', 'NN')], [('the', 'DT'), ('yacht', 'NN')], [('whitney', 'NN')], [('a', 'DT'), ('laugh', 'NN')], [('i', 'NN')], [('a', 'DT'), ('moose', 'NN')]]


In [16]:
for a_list in NPleaves[0:20]:
    np = [word for word,tag in a_list]
    phrase = " ".join(np)
    print(phrase)

the right
a large island
whitney
a mystery
island
ship trap island
whitney
a suggestive name
isn
a curious dread
the place
some superstition
rainsford
the dank tropical night
thick warm blackness
the yacht
whitney
a laugh
i
a moose


### Named Entities

You will need to download the named entity chunker first: `nltk.download("maxent_ne_chunker")`.

For more on the kinds of named entities: https://www.nltk.org/book/ch07.html#sec-ner.

In [ ]:
tree = nltk.ne_chunk(mdg_tagged, binary=True)

def extract_ne(text):
    # tokenize by word
    words = nltk.tokenize.word_tokenize(text)
    # apply part of speech tags to those words
    tags = nltk.pos_tag(words)
    # extract named entities based on those tags
    # "binary=True ==> named entities won’t be labeled by kind
    tree = nltk.ne_chunk(tags, binary=True)
    ne_set = set(
        " ".join(i[0] for i in t)
        for t in tree
        if hasattr(t, "label") and t.label() == "NE"
    )
    return ne_set

In [ ]:
mdg_ne = extract_ne(mdg)
print(mdg_ne)

{'Amazon', 'Rio', 'America', 'Rest', 'New York', 'Puritan', 'English', 'Caucasus', 'Island', 'American', 'Great White Czar', 'Uganda', 'Turkish', 'Malay', 'Rainsford', 'Suzette', 'Lazarus', 'Moscow', 'Russia', 'Africa', 'Ganges', 'General Zaroff', 'Malacca', 'Tibet', 'Purdey', 'Marcus Aurelius', 'Dusk', 'Caribbean', 'Mr. Rainsford', 'Caribbean Sea', 'Sleep', 'San Lucar', 'East Africa', 'God', 'Spanish', 'Pol Roger', 'Ship Trap', 'Sanger Rainsford', 'Chambertin', 'Chinese', 'Monte Carlo', 'Night', 'Crimea', 'Ivan', 'Madame Butterfly', 'Zaroff', 'Captain Nielsen', 'Toward', 'France', 'New York City', 'London', 'Rains', 'Whitney', 'Mr. Sanger Rainsford', 'French', 'Chablis', 'Veuve Cliquot', 'Russian', 'Great', 'Cossack', 'Paris'}
